In [1]:
import pandas as pd

import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)

<module 'model.modelAI' from 'D:\\Proyectos\\OSMOSYS_2022\\IntegraciónAI\\AI_OsmosysIntegrationScripts\\model\\modelAI.py'>

In [2]:
## parameters
indicatorCodeAI = 'S8_01'
month = 'SEPTIEMBRE'
month_number = '2022-09'
year = 2022
test = True
print(
    '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')


--------------------------------------S8_01------------------------------------------------------


In [5]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
print(subformDf)
indicatorIdsOsmosys = subformDf.osmosys_indicador_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicator_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

Empty DataFrame
Columns: [folder, form, form_id, parent_form, parent_form_id, actividad, codigo, indicador, indicator_id, osmosys_indicador_id, osmosys_indicador_code, osmosys_indicador, osmosys_indicador_categoria]
Index: []


IndexError: single positional indexer is out-of-bounds

In [20]:
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))
## consulto los datos reportados
reportedDf = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                                month=month, year=year)
print('reported data: ' + str(reportedDf.shape[0]))


indicatorIdsOsmosys: [101]
reported data: 1


In [21]:
## obtengo los parents de AI
parentFormsIds = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf)

In [23]:
## contruyo las estructuras de datos
changesList = []
newIds = []
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)

    poblacion_meta = ["Refugiados/as y migrantes", "Comunidad de acogida"]

    S8_01_RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    S8_01_RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    S8_01_RM_MM = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    S8_01_RM_HH = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    S8_01_RM_OTR = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)


    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)

    S8_01_CA_NA = int(dfCa.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    S8_01_CA_NN = int(dfCa.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    S8_01_CA_MM = int(dfCa.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    S8_01_CA_HH = int(dfCa.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    S8_01_CA_OTR = int(dfCa.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                   cantonCode=cantonCode)

    # print(dfDiversidad)
    S8_01_RM_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
                dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    S8_01_CA_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
                dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)



    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                                   indicatorsIdsOmosys=indicatorIdsOsmosys).iloc[
            0].value_a

    subform = model.modelAI.SubFormS8_01(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        covid='No',
        poblacion_meta=poblacion_meta,
        S8_01_RM_NA=S8_01_RM_NA,
        S8_01_RM_NN=S8_01_RM_NN,
        S8_01_RM_MM=S8_01_RM_MM,
        S8_01_RM_HH=S8_01_RM_HH,
        S8_01_RM_OTR=S8_01_RM_OTR,
        S8_01_RM_LGBT=S8_01_RM_LGBT,
        S8_01_CA_NA=S8_01_CA_NA,
        S8_01_CA_NN=S8_01_CA_NN,
        S8_01_CA_MM=S8_01_CA_MM,
        S8_01_CA_HH=S8_01_CA_HH,
        S8_01_CA_OTR=S8_01_CA_OTR,
        S8_01_CA_LGBT=S8_01_CA_LGBT,
        S8_01_CUAL=commentary
    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))

changes to charge: 1


In [24]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)

In [25]:
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month, "data_" + indicatorCodeAI + '-' + str(indicatorIdsOsmosys) + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: " + text_file.name)
newIdsDict = {"newIds": newIds}
newIdsDf = pd.DataFrame(newIdsDict)
newIdsDf.to_csv(os.path.join(month, "new_ids_" + indicatorCodeAI + ".csv"))

 creado respaldo: AGOSTO\data_S8_01-[101].json


In [26]:
## envio a AI
if (test):
    print(
        '--------------------------------------' + indicatorCodeAI + '-' + str(
            indicatorIdsOsmosys) + '------------------------------------------------------')
else:
    try:
        if len(changesList) > 0:
            print('se envia a AI')
            client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
            client.post_resource(path='update', body=finalJson)
            print(
                '--------------------------------------' + indicatorCodeAI + '-' + str(
                    indicatorIdsOsmosys) + '------------------------------------------------------')
        else:
            print('nada que enviar a AI')
    except HTTPError as e:
        code = e.response.status_code
        print('error:')
        print(code)

        print(e)

--------------------------------------S8_01-[101]------------------------------------------------------
